# Tutorial 1: Anomalous Diffusion and Fractional Dynamics

This notebook demonstrates how to simulate and analyze anomalous diffusion using Fractional Brownian Motion (fBm).

## 1. Introduction

Anomalous diffusion is characterized by a non-linear theoretical Mean Squared Displacement (MSD):

$$\langle x^2(t)\rangle \propto t^\alpha$$

- $\alpha < 1$: Subdiffusion (hindered movement)
- $\alpha = 1$: Normal Brownian Motion
- $\alpha > 1$: Superdiffusion (active transport/turbulence)

## 2. Generating fBm

We use spectral synthesis to generate fractional Brownian motion.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

def generate_fbm(n_samples, hurst):
    beta = 2 * hurst + 1
    freqs = np.fft.rfftfreq(n_samples * 2)
    magnitude = np.zeros_like(freqs)
    magnitude[1:] = freqs[1:] ** (-beta / 2.0)
    phase = np.random.uniform(0, 2*np.pi, size=len(freqs))
    coeffs = magnitude * np.exp(1j * phase)
    fgn = np.fft.irfft(coeffs)
    fbm = np.cumsum(fgn[:n_samples])
    return fbm / np.std(fbm)

# Plot
plt.figure(figsize=(10, 6))
t = np.linspace(0, 1, 1000)
plt.plot(t, generate_fbm(1000, 0.3), label='H=0.3 (Sub)')
plt.plot(t, generate_fbm(1000, 0.5), label='H=0.5 (Normal)')
plt.plot(t, generate_fbm(1000, 0.7), label='H=0.7 (Super)')
plt.legend()
plt.title("Fractional Brownian Motion Trajectories")
plt.grid(True)
plt.show()


## 3. MSD Analysis

We calculate the MSD to estimate the exponent $\alpha$.

In [ ]:

def calculate_msd(trajectory, max_lag=200):
    msd = np.zeros(max_lag)
    for lag in range(1, max_lag + 1):
        diff = trajectory[lag:] - trajectory[:-lag]
        msd[lag-1] = np.mean(diff ** 2)
    return np.arange(1, max_lag + 1), msd

# Analyze H=0.7 case
traj = generate_fbm(2000, 0.7)
lags, msd = calculate_msd(traj)

# Fit power law
log_lags = np.log(lags)
log_msd = np.log(msd)
slope, _ = np.polyfit(log_lags, log_msd, 1)

plt.figure(figsize=(8, 6))
plt.loglog(lags, msd, label=f'Data (Slope={slope:.2f})')
plt.loglog(lags, lags**(2*0.7), '--', label='Theoretical (Slope=1.4)')
plt.legend()
plt.title("MSD Analysis")
plt.xlabel("Lag")
plt.ylabel("MSD")
plt.grid(True, which="both")
plt.show()
